# Setup


In [ ]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
load_dotenv()

import sys
sys.path.append('./src')


# Tools

## Composio

In [ ]:
from composio import Composio

composio_client = Composio()

entity = composio_client.get_entity()

print(entity.get_connections())

In [20]:
actions = composio_client.actions.get()


In [ ]:

# filter elements of actions array where attribute name starts with 'GOOGLE 
google_actions = list(filter(lambda action: action.name.startswith('GOOGLEDOCS'), actions))


# pretty print actions, ActionModel elements
for action in google_actions:
    print(action)


In [ ]:
from composio_crewai import ComposioToolSet, Action
from composio.utils import logging

logging.setup(logging.LogLevel.DEBUG)
tool_set = ComposioToolSet()
tools = tool_set.get_tools(actions=[Action.GOOGLEDOCS_UPDATE_EXISTING_DOCUMENT, Action.GOOGLEDOCS_GET_DOCUMENT_BY_ID])

In [ ]:
tools

## Scrapting Fish

In [6]:
from agents.tool_evaluator.tools import ScrapingFishTool

In [20]:
import os
SCRAPING_FISH_API_KEY = os.getenv("SCRAPING_FISH_API_KEY")


In [ ]:
content = ScrapingFishTool().run('https://www.orulo.com.br')

# Models

## Google Gemini

In [ ]:
import os
from google.oauth2 import service_account
from langchain_google_genai import ChatGoogleGenerativeAI

# Set the path to the service account JSON key file
service_account_file = os.getenv("GOOGLE_SERVICE_ACCOUNT_FILE")

# Authenticate using the service account key
credentials = service_account.Credentials.from_service_account_file(service_account_file)


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash',
                            verbose=True,
                            temperature=0.5,
                            credentials=credentials)

In [ ]:
print(type(llm))

In [ ]:
isinstance(llm, ChatGoogleGenerativeAI)

In [ ]:
llm.model.split('/')[-1]

In [ ]:
llm.invoke("Sing a ballad of LangChain.")

## OpenAI

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o',
                  verbose=True,
                  temperature=0)

In [ ]:
?ChatOpenAI

## LiteLLM

In [ ]:
import os
from google.oauth2 import service_account
from langchain_google_genai import ChatGoogleGenerativeAI

# Set the path to the service account JSON key file
service_account_file = os.getenv("GOOGLE_SERVICE_ACCOUNT_FILE")

# Authenticate using the service account key
credentials = service_account.Credentials.from_service_account_file(service_account_file)

In [6]:
from langchain_community.chat_models import ChatLiteLLM
chat = ChatLiteLLM(model="gpt-3.5-turbo")

In [ ]:
type(chat)

In [15]:
chat.model_name = chat.model

In [ ]:
chat.model_name

In [ ]:
hasattr(chat, "model_name")

## Ollama

In [53]:
from langchain_community.llms import Ollama

llm = Ollama(model='invalid')

## LLM Wrapper


In [ ]:
# print current directory
import os

print(os.getcwd())

In [19]:
# add ./src to python path
import sys
sys.path.append('./src')


In [20]:
from helpers.llm_wrapper import ModelParams

p = ModelParams()


In [ ]:
p.use_cache

## Trulens

In [16]:
from helpers.llm_wrapper import ModelParams

p = ModelParams(use_trulens=True)

In [ ]:
p.use_trulens

In [ ]:
from helpers.llm_wrapper import get_llm, get_model_name
llm = get_llm('openai', 'gpt-3.5-turbo-1106', config=p)
print(get_model_name(llm))

# Observability


## Helicone

In [33]:
params = {
    "model": 'gpt-3.5-turbo',
    "temperature": 0,
    "verbose": True,
}

params["base_url"] = "https://oai.helicone.ai/v1"

In [34]:
import os
headers = {
    "Helicone-Auth": f"Bearer {os.getenv('HELICONE_API_KEY')}",
}

In [ ]:
headers

In [36]:
params["default_headers"] = headers

In [ ]:
params

In [49]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(**params)

In [ ]:
llm.invoke("Sing a ballad of LangChain.")

In [40]:
from helpers.tracer import TracerFactory

tracer = TracerFactory.get_tracer()

In [42]:
from helpers.tracer import TracerAnnotation
tracer.provider = 'helicone'
tracer.annotation = TracerAnnotation(app='tool_evaluator_agent')

In [ ]:
proxy_config = tracer.get_proxy_config()
proxy_config

In [ ]:
params

In [47]:
params["base_url"] = proxy_config.url
params["default_headers"] = proxy_config.headers

In [ ]:
__name__

In [7]:
## Tracer

In [8]:
from helpers.tracer import TracerFactory, TracerAnnotation
tracer = TracerFactory.get_tracer()

In [ ]:
tracer

In [7]:
tracer.provider = 'helicone'

In [ ]:
tracer

In [ ]:
tracer.get_proxy_config()

## Logger

In [5]:
import logging

In [7]:
logger = logging.getLogger('tool-evaluator-agent')

In [ ]:
provider = "Google"
logger.debug(f"Tracer ended with provider: {provider}")

In [ ]:
# Configure the root logger
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)


In [ ]:
logger = logging.getLogger('tool-evaluator-agent')
provider = "Google"
logger.debug(f"Tracer ended with provider: {provider}")

In [12]:

# Create a common logger instance
logger = logging.getLogger('tool-evaluator-agent')

# Example: Adding a file handler
file_handler = logging.FileHandler('logs/app-debug.log')
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(
    logging.Formatter('%(asctime)s - %(name)s - '
                      '%(levelname)s - %(message)s'))
logger.addHandler(file_handler)

In [ ]:
logger = logging.getLogger('tool-evaluator-agent')
provider = "Google"
logger.debug(f"Tracer ended with provider: {provider}")

In [ ]:
type(logger)

In [ ]:
print(logger)

# Agents & Tasks

## Fedback Functions

In [ ]:
import numpy as np
from trulens.core import Feedback
from trulens.providers.openai import OpenAI
from trulens.providers.bedrock import Bedrock
from trulens.providers.bedrock.endpoint import BedrockEndpoint
#import boto3

# Initialize provider class
provider = OpenAI(model_engine='gpt-4o')

# boto3_session = boto3.Session(profile_name='iaproject', region_name='us-west-2')
# boto3_client = boto3_session.client('bedrock')
# provider = Bedrock(model_id='anthropic.claude-3-opus-20240229-v1:0', client=boto3_client, region_name='us-west-2')

# select context to be used in feedback. the location of context is app specific.
# context = TruChain.select_context(rag_chain)

# Define a groundedness feedback function
# f_groundedness = (
#     Feedback(
#         provider.groundedness_measure_with_cot_reasons, name="Groundedness"
#     )
#     .on(context.collect())  # collect context chunks into a list
#     .on_output()
# )

# Question/answer relevance between overall question and answer.
f_answer_relevance = Feedback(
     provider.relevance_with_cot_reasons, name="Answer Relevance"
).on_input_output()

# Context relevance between question and each context chunk.
# f_context_relevance = (
#     Feedback(
#         provider.context_relevance_with_cot_reasons, name="Context Relevance"
#     )
#     .on_input()
#     .on(context)
#     .aggregate(np.mean)
# )

In [ ]:
boto3_session

In [ ]:
provider

## Tracer

In [4]:
from helpers.tracer import TracerFactory, TracerAnnotation
tracer = TracerFactory.get_tracer()
tracer.provider = 'helicone'


## Tool Evaluator

In [ ]:
from agents.tool_evaluator.crew import ToolEvalCrewFactory
from agents.tool_evaluator.main import crew_inputs as te_crew_inputs
from trulens.apps.langchain import TruChain
from trulens.core import TruSession
from trulens.apps.langchain import TruChain

te_factory = ToolEvalCrewFactory()

te_session = TruSession()
#te_session.reset_database()

te_crew = te_factory.crew(agents=[te_factory.researcher()], 
                          tasks=[te_factory.criteria_clarification_task()])

te_true_app = TruChain(te_crew.agents[0],
                       app_name=f"researcher@criteria-clarification-task",
                       app_version="0.0.1",
                       feedbacks=[f_answer_relevance])

tracer.annotation = TracerAnnotation(app=te_true_app.app_name)
print(f"Tracer annotation is {tracer.annotation}")

with te_true_app as te_recordings:    
    tracer.init()
    te_crew.kickoff(te_crew_inputs)
    tracer.end()

In [ ]:
from tool_evaluator_agent.main import crew_inputs

crew = factory.crew(agents=[factory.researcher()], tasks=[factory.research_task()])

from trulens.apps.langchain import TruChain
true_app = TruChain(crew.agents[0],
                    app_name=f"researcher@research-task",
                    app_version="0.0.1",
                    feedbacks=[f_answer_relevance])

with true_app as recordings:    
    crew.kickoff(crew_inputs)

In [ ]:
te_session.get_leaderboard()

In [ ]:
from trulens.dashboard.display import get_feedback_result

te_last_record = te_recordings.records[-1]
te_feedback_df = get_feedback_result(te_last_record, "Answer Relevance")


# align text to the left and do not crop cell content
te_feedback_df.head(1).T.style.set_properties(**{'text-align': 'left', 'white-space': 'pre-wrap'})

In [ ]:
te_records, te_feedback = te_session.get_records_and_feedback()

te_records.tail()

In [ ]:
te_records.tail(1)['Answer Relevance_calls'].values[0][0]['args']['prompt']

In [ ]:
provider

In [ ]:
te_records.tail(1)['Answer Relevance_calls'].values[0][0]['args']['prompt']

In [ ]:
te_records.tail(1)['Answer Relevance_calls'].values[0][0]['args']['response']

In [ ]:
provider.relevance_with_cot_reasons(te_records.tail(1)['Answer Relevance_calls'].values[0][0]['args']['prompt'],
                                    te_records.tail(1)['Answer Relevance_calls'].values[0][0]['args']['response'])


In [ ]:
json_like = last_record.layout_calls_as_app()

from ipytree import Node
from ipytree import Tree


def display_call_stack(data):
    tree = Tree()
    tree.add_node(Node("Record ID: {}".format(data["record_id"])))
    tree.add_node(Node("App ID: {}".format(data["app_id"])))
    tree.add_node(Node("Cost: {}".format(data["cost"])))
    tree.add_node(Node("Performance: {}".format(data["perf"])))
    tree.add_node(Node("Timestamp: {}".format(data["ts"])))
    tree.add_node(Node("Tags: {}".format(data["tags"])))
    tree.add_node(Node("Main Input: {}".format(data["main_input"])))
    tree.add_node(Node("Main Output: {}".format(data["main_output"])))
    tree.add_node(Node("Main Error: {}".format(data["main_error"])))

    calls_node = Node("Calls")
    tree.add_node(calls_node)

    for call in data["calls"]:
        call_node = Node("Call")
        calls_node.add_node(call_node)

        for step in call["stack"]:
            step_node = Node("Step: {}".format(step["path"]))
            call_node.add_node(step_node)
            if "expanded" in step:
                expanded_node = Node("Expanded")
                step_node.add_node(expanded_node)
                for expanded_step in step["expanded"]:
                    expanded_step_node = Node(
                        "Step: {}".format(expanded_step["path"])
                    )
                    expanded_node.add_node(expanded_step_node)

    return tree


# Usage
tree = display_call_stack(json_like)
tree

In [ ]:
from trulens.dashboard import run_dashboard

run_dashboard(te_session)

In [ ]:
from trulens.dashboard import run_dashboard

run_dashboard(te_session)

## Feature Formulator

In [ ]:
from agents.feature_formulator.crew import FeatureFormulatorCrewFactory
from agents.feature_formulator.main import crew_inputs as ff_crew_inputs
from trulens.apps.langchain import TruChain
from trulens.core import TruSession
from trulens.apps.langchain import TruChain


ff_factory = FeatureFormulatorCrewFactory()

ff_session = TruSession()
#ff_session.reset_database()


In [ ]:

ff_crew = ff_factory.crew(agents=[ff_factory.product_owner()], 
                          tasks=[ff_factory.feature_specification_task()])

ff_true_app = TruChain(ff_crew.agents[0],
                       app_name=f"product_owner@feature_specification_task",
                       app_version="0.0.1",
                       feedbacks=[f_answer_relevance])

tracer.annotation = TracerAnnotation(app=ff_true_app.app_name)
print(f"Tracer annotation is {tracer.annotation}")

with ff_true_app as ff_recordings:    
    tracer.init()
    ff_crew.kickoff(ff_crew_inputs)
    tracer.end()


In [ ]:
ff_crew = ff_factory.crew(agents=[ff_factory.product_owner()], 
                          tasks=[ff_factory.previous_spec_review_task()])

ff_true_app = TruChain(ff_crew.agents[0],
                       app_name=f"teste",
                       app_version="0.0.1",
                       feedbacks=[f_answer_relevance])

tracer.annotation = TracerAnnotation(app=ff_true_app.app_name)
print(f"Tracer annotation is {tracer.annotation}")

with ff_true_app as ff_recordings:    
    tracer.init()
    ff_crew.kickoff(ff_crew_inputs)
    tracer.end()

In [ ]:
from trulens.dashboard import run_dashboard

run_dashboard(ff_session)